In [1]:
import sys
import dagger
from dagger import dag

In [2]:
from workspace import Workspace

In [3]:
SQL_SCRIPT = """
create schema api;

create table api.todos (
  id int primary key generated by default as identity,
  done boolean not null default false,
  task text not null,
  due timestamptz
);

insert into api.todos (task) values
  ('finish tutorial 0'), ('pat self on back');


create role web_anon nologin;

grant usage on schema api to web_anon;
grant select on api.todos to web_anon;
"""

In [4]:
async with dagger.connection(dagger.Config(log_output=sys.stderr)):
    workspace = await Workspace.create()
    result = await workspace.exec_all(
        sql_files=[SQL_SCRIPT],
        command=["curl", "http://postgrest:3000/todos"],
    )


Output()

In [5]:
print(result.stdout)

[{"id":1,"done":false,"task":"finish tutorial 0","due":null}, 
 {"id":2,"done":false,"task":"pat self on back","due":null}]
